In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [4]:
items_weights = data_train.groupby('item_id').agg(sales=('sales_value', 'sum')).reset_index()
items_weights['weights'] = items_weights[['sales']].apply(lambda x: x / np.sum(x))
items_weights

,item_id,sales,weights
0,25671,20.94,2.969296e-06
1,26081,0.99,1.403822e-07
2,26093,1.59,2.254623e-07
3,26190,1.54,2.183723e-07
4,26355,1.98,2.807644e-07
...,...,...,...
86860,17381856,0.00,0.000000e+00
86861,17382205,7.99,1.132983e-06
86862,17383227,4.49,6.366828e-07
86863,17827644,2.50,3.545005e-07


In [5]:
def weighted_random_recommendation(items_weights, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """

    items = np.array(items_weights['item_id'])
    items_weights = np.array(items_weights['weights'])
    recs = np.random.choice(items, size=n, replace=False)
    
    
    return recs.tolist()

In [6]:
%%time
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']

result['weighted_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_weights, n=5))

result.head(2)

CPU times: user 16.2 s, sys: 27.3 ms, total: 16.2 s
Wall time: 31 s


,user_id,actual,weighted_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[12158420, 10283062, 13506702, 12384999, 1010481]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[15715861, 10121984, 121520, 1082311, 1129536]"


In [7]:
def precision_at_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [8]:
for name_col in result.columns[1:]:
#     print(name_col)
    print(f"{round(result.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0007:weighted_recommendation


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [9]:
result1 = pd.read_csv('predictions_basic.csv')
result1.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[482616, 12985761, 848987, 15973193, 1111296]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1127831, 995242]","[1082185, 6666, 981760, 1127831, 1098066]","[1082185, 981760, 1127831, 6666, 1098066]","[1082185, 995242, 1029743, 840361, 904360]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[5691060, 15716232, 157463, 1092311, 873474]","[6534178, 6533889, 1029743, 6534166, 1082185]","[6666, 1082185, 981760, 1098066, 995242]","[1082185, 1098066, 981760, 6666, 826249]","[1082185, 981760, 1098066, 826249, 6666]","[1082185, 1098066, 6534178, 826249, 1127831]"


In [10]:
result1.actual = np.array(result1.actual.str.strip('[]').str.split().apply(pd.to_numeric))

for col in result1.columns.values[2:]:
    result1[col] = np.array(result1[col].str.strip('[]').str.split(',').apply(pd.to_numeric))

In [11]:
for name_col in result1.columns[1:]:
#     print(name_col)
    print(f"{round(result1.apply(lambda row: precision_at_k(row[name_col], row['actual']), axis=1).mean(),4)}:{name_col}")

1.0:actual
0.0002:random_recommendation
0.1552:popular_recommendation
0.1368:itemitem
0.1329:cosine
0.139:tfidf
0.2193:own_purchases


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [12]:
# your_code